In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 131352 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.1-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=ht

In [34]:
!mkdir -p drive
!google-drive-ocamlfuse drive

fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option


In [0]:
!ls "/content/drive/CNNModel"


clean_tweet.csv
deep_learning_tweets.ipynb
part11.ipynb
testdata.manual.2009.06.14.csv
w2v_model_ug_cbow.word2vec
w2v_model_ug_cbow.word2vec.trainables.syn1neg.npy
w2v_model_ug_cbow.word2vec.wv.vectors.npy
w2v_model_ug_sg.word2vec
w2v_model_ug_sg.word2vec.trainables.syn1neg.npy
w2v_model_ug_sg.word2vec.wv.vectors.npy


In [0]:
import pandas as pd  
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [0]:
my_df = pd.read_csv("/content/drive/CNNModel/clean_tweet.csv",index_col=0)
my_df.head()

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,text,target
0,awww that bummer you shoulda got david carr of...,0
1,is upset that he can not update his facebook b...,0
2,pre stress stressed like stressed about the st...,0
3,cannot attend the concert this time have too m...,0
4,no it not behaving at all mad why am here beca...,0


In [0]:
my_df.dropna(inplace=True)
my_df.reset_index(drop=True,inplace=True)
my_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1596019 entries, 0 to 1596018
Data columns (total 2 columns):
text      1596019 non-null object
target    1596019 non-null int64
dtypes: int64(1), object(1)
memory usage: 24.4+ MB


In [0]:
x = my_df.text
y = my_df.target

In [0]:
from sklearn.model_selection import train_test_split
SEED = 2000
x_train, x_validation_and_test, y_train, y_validation_and_test = train_test_split(x, y, test_size=.02, random_state=SEED)
x_validation, x_test, y_validation, y_test = train_test_split(x_validation_and_test, y_validation_and_test, test_size=.5, random_state=SEED)

In [0]:
!pip install sklearn

In [0]:
print ("Train set has total {0} entries with {1:.2f}% negative, {2:.2f}% positivity".format(len(x_train),
                                                                             (len(x_train[y_train == 0]) / (len(x_train)*1.))*100,
                                                                            (len(x_train[y_train == 1]) / (len(x_train)*1.))*100))
print ("Validation set has total {0} entries with {1:.2f}% negative, {2:.2f}% positive".format(len(x_validation),
                                                                             (len(x_validation[y_validation == 0]) / (len(x_validation)*1.))*100,
                                                                            (len(x_validation[y_validation == 1]) / (len(x_validation)*1.))*100))
print ("Test set has total {0} entries with {1:.2f}% negative, {2:.2f}% positive".format(len(x_test),
                                                                             (len(x_test[y_test == 0]) / (len(x_test)*1.))*100,
                                                                            (len(x_test[y_test == 1]) / (len(x_test)*1.))*100))

Train set has total 1564098 entries with 50.00% negative, 50.00% positivity
Validation set has total 15960 entries with 50.40% negative, 49.60% positive
Test set has total 15961 entries with 50.26% negative, 49.74% positive


In [0]:
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
import gensim
from gensim.models.word2vec import Word2Vec
from gensim.models.doc2vec import TaggedDocument
import multiprocessing
from sklearn import utils

In [0]:
def labelize_tweets_ug(tweets,label):
    result = []
    prefix = label
    for i, t in zip(tweets.index, tweets):
        result.append(TaggedDocument(t.split(), [prefix + '_%s' % i]))
    return result


In [0]:
all_x = pd.concat([x_train,x_validation,x_test])
all_x_w2v = labelize_tweets_ug(all_x, 'all')

In [0]:
from gensim.models import KeyedVectors
model_ug_cbow = KeyedVectors.load('/content/drive/CNNModel/w2v_model_ug_cbow.word2vec')
model_ug_sg = KeyedVectors.load('/content/drive/CNNModel/w2v_model_ug_sg.word2vec')

In [0]:
len(model_ug_cbow.wv.vocab.keys())

106257

In [0]:
embeddings_index = {}
for w in model_ug_cbow.wv.vocab.keys():
    embeddings_index[w] = np.append(model_ug_cbow.wv[w],model_ug_sg.wv[w])
print('Found %s word vectors.' % len(embeddings_index))

Found 106257 word vectors.


In [0]:
!pip install Keras

In [0]:
import keras
keras.__version__

Using TensorFlow backend.


'2.2.4'

In [0]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=100000)
tokenizer.fit_on_texts(x_train)
sequences = tokenizer.texts_to_sequences(x_train)

In [0]:
len(tokenizer.word_index)

261649

In [0]:
for x in x_train[:5]:
    print (x)

hate you
got physics exam jealous
holy shit new vending machine blue lcd bit but no fun carousel delivery system
ha no went to real gym swear look what found of our fave things chocolate nathan
loved my doggie kisses made my day can not tell little bear tho


In [0]:
sequences[:5]

[[137, 6],
 [45, 2569, 410, 555],
 [1311, 353, 69, 9569, 1496, 855, 9411, 235, 17, 35, 104, 18470, 2871, 1443],
 [384,
  35,
  203,
  1,
  372,
  629,
  1491,
  202,
  46,
  302,
  11,
  177,
  1418,
  223,
  673,
  4994],
 [502, 3, 4487, 2134, 195, 3, 29, 26, 7, 253, 157, 1758, 329]]

In [0]:
length = []
for x in x_train:
    length.append(len(x.split()))

In [0]:

max(length)

40

In [0]:

x_train_seq = pad_sequences(sequences, maxlen=45)
print('Shape of data tensor:', x_train_seq.shape)

Shape of data tensor: (1564098, 45)


In [0]:
x_train_seq[:5]

array([[    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,   137,     6],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,    45,  2569,   410,   555],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,  1311,   353,    69,  9569,  1496,
    

In [0]:
sequences_val = tokenizer.texts_to_sequences(x_validation)
x_val_seq = pad_sequences(sequences_val, maxlen=45)

In [0]:
num_words = 100000
embedding_matrix = np.zeros((num_words, 200))
for word, i in tokenizer.word_index.items():
    if i >= num_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [0]:
np.array_equal(embedding_matrix[6] ,embeddings_index.get('you'))

True

In [0]:
from keras.layers import Input, Dense, concatenate, Activation
from keras.models import Model
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D

tweet_input = Input(shape=(45,), dtype='int32')

tweet_encoder = Embedding(100000, 200, weights=[embedding_matrix], input_length=45, trainable=True)(tweet_input)
bigram_branch = Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1)(tweet_encoder)
bigram_branch = GlobalMaxPooling1D()(bigram_branch)
trigram_branch = Conv1D(filters=100, kernel_size=3, padding='valid', activation='relu', strides=1)(tweet_encoder)
trigram_branch = GlobalMaxPooling1D()(trigram_branch)
fourgram_branch = Conv1D(filters=100, kernel_size=4, padding='valid', activation='relu', strides=1)(tweet_encoder)
fourgram_branch = GlobalMaxPooling1D()(fourgram_branch)
merged = concatenate([bigram_branch, trigram_branch, fourgram_branch], axis=1)

merged = Dense(256, activation='relu')(merged)
merged = Dropout(0.2)(merged)
merged = Dense(1)(merged)
output = Activation('sigmoid')(merged)
model = Model(inputs=[tweet_input], outputs=[output])
model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 45)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 45, 200)      20000000    input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 44, 100)      40100       embedding_1[0][0]                
__________________________________________________________________________________________________
conv1d_2 (Conv1D)    

In [0]:
from keras.callbacks import ModelCheckpoint

filepath="CNN_best_weights.{epoch:02d}-{val_acc:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

model.fit(x_train_seq, y_train, batch_size=32, epochs=5,
                     validation_data=(x_val_seq, y_validation), callbacks = [checkpoint])


Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 1564098 samples, validate on 15960 samples
Epoch 1/5
1564098/1564098 [==============================] - 1483s 948us/step - loss: 0.4033 - acc: 0.8182 - val_loss: 0.3808 - val_acc: 0.8288

Epoch 00001: val_acc improved from -inf to 0.82882, saving model to CNN_best_weights.01-0.8288.hdf5
Epoch 2/5
1564098/1564098 [==============================] - 1476s 944us/step - loss: 0.3657 - acc: 0.8396 - val_loss: 0.3788 - val_acc: 0.8343

Epoch 00002: val_acc improved from 0.82882 to 0.83427, saving model to CNN_best_weights.02-0.8343.hdf5
Epoch 3/5
 556448/1564098 [=========>....................] - ETA: 16:11 - loss: 0.3322 - acc: 0.8571Buffered data was truncated after reaching the output size limit.

In [0]:
from keras.models import load_model
loaded_CNN_model = load_model('CNN_best_weights.01-0.8215.hdf5')
loaded_CNN_model.evaluate(x=x_val_seq, y=y_validation)

OSError: ignored

Saving model to drive

In [0]:
model_json = model.to_json()
with open("drive/CNNModel/model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("drive/CNNModel/model_weights.h5")
print("Saved model to drive")

testing the model

In [0]:
sequences_test = tokenizer.texts_to_sequences(x_test)
x_test_seq = pad_sequences(sequences_test, maxlen=45)

In [0]:

loaded_CNN_model.evaluate(x=x_test_seq, y=y_test)

trying the predict method

In [0]:
for x in x_test_seq[:5]:
    print (x)

In [0]:
loaded_CNN_model.predict(x_test_seq)